In [105]:
import nltk
import pandas as pd
import re
import nltk
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import numpy as np

In [41]:
# path of the given airline dataset
pathairline = "Tweets.csv"

In [43]:

df_of_tweets = pd.read_csv(pathairline)


In [44]:
df_of_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [45]:
df_of_tweets['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

## Vectorizer and Stopword removal

In [47]:
negative_word_doc = df_of_tweets['text'].loc[df_of_tweets['airline_sentiment']=='negative']
negative_word_doc[:5]

3     @VirginAmerica it's really aggressive to blast...
4     @VirginAmerica and it's a really big bad thing...
5     @VirginAmerica seriously would pay $30 a fligh...
15        @VirginAmerica SFO-PDX schedule is still MIA.
17    @VirginAmerica  I flew from NYC to SFO last we...
Name: text, dtype: object

In [48]:
positive_word_doc = df_of_tweets['text'].loc[df_of_tweets['airline_sentiment']=='positive']
positive_word_doc[:5]

1     @VirginAmerica plus you've added commercials t...
6     @VirginAmerica yes, nearly every time I fly VX...
8       @virginamerica Well, I didn't…but NOW I DO! :-D
9     @VirginAmerica it was amazing, and arrived an ...
11    @VirginAmerica I &lt;3 pretty graphics. so muc...
Name: text, dtype: object

In [49]:
#Removing stopwords and punctuations from tweets

In [51]:
new_stopwords = []
stopwords = stopwords.words('english')
stopwords.extend(new_stopwords)

In [53]:
# regex to remove numbers
expression = re.compile(r'\d')
negative_word_doc = pd.Series([re.sub(expression, '', word) for word in negative_word_doc])
positive_word_doc = pd.Series([re.sub(expression, '', word) for word in positive_word_doc])

In [54]:
# remove stopwords url and @mentions from tweets
negative_word_doc = negative_word_doc.apply(lambda x: ' '.join([word.lower() for word in x.split() if word.find('@') == -1 and \
                                                                word not in stopwords and \
                                                                word.find('http') == -1]))



In [55]:
positive_word_doc = positive_word_doc.apply(lambda x: ' '.join([word.lower() for word in x.split() if word.find('@') == -1 and \
                                                                word not in stopwords and \
                                                                word.find('http') == -1]))

In [57]:
negative_word_doc[:7]

0    really aggressive blast obnoxious "entertainme...
1                                 really big bad thing
2    seriously would pay $ flight seats playing. re...
3                          sfo-pdx schedule still mia.
4    flew nyc sfo last week fully sit seat due two ...
5    first fares may three times carriers seats ava...
6    guys messed seating.. reserved seating friends...
dtype: object

In [60]:
#Top 50 adjectives in positive tweets
adj_words = []
for doc in positive_word_doc:
    for word in word_tokenize(doc):
        pos = wn.synsets(word, pos='a')
        if len(pos) > 0:
            adj_words.append(word)

pos_adj_words = pd.Series(adj_words).value_counts()[:50]

# print all words in positive tweets separated by comma and in quotes
print(', '.join(["'{}'".format(word) for word in pos_adj_words.index]))

'great', 'i', 'much', 'good', 'best', 'awesome', 'amazing', 'back', 'flying', 'fly', 'made', 'like', 'home', 'new', 'plane', 'first', 'one', 'well', 'nice', 'u', 'still', 'helpful', 'go', 'cancelled', 'better', 'next', 'southwest', 'last', 'sent', 'happy', 'weather', 'free', 'quick', 'done', 'attendant', 'even', 'united', 'right', 'able', 'sure', 'big', 'another', 'looking', 'up', 'forward', 'ok', 'excellent', 'wonderful', 'cool', 'glad'


In [61]:
#Top 50 adjectives in negative tweets
adj_words = []
for doc in negative_word_doc:
    for word in word_tokenize(doc):
        pos = wn.synsets(word, pos='a')
        if len(pos) > 0:
            adj_words.append(word)

neg_adj_words = pd.Series(adj_words).value_counts()[:50]

# print all words in negative tweets separated by comma and in quotes
print(', '.join(["'{}'".format(word) for word in neg_adj_words.index]))

'i', 'cancelled', 'plane', 'delayed', 'still', 'one', 'late', 'back', 'waiting', 'like', 'trying', 'going', 'even', 'u', 'worst', 'fly', 'last', 'lost', 'another', 'weather', 'due', 'united', 'flighted', 'go', 'home', 'sitting', 'two', 'flying', 'first', 'bad', 'stuck', 'online', 'said', 'next', 'good', 'new', 'right', 'made', 'better', 'rude', 'long', 'tried', 'missed', 'left', 'booked', 'sure', 'ago', 'on', 'terrible', 'sent'


In [62]:
def document_features(document, word_features):
	document_words = set(document)
	features = {}
	for word in word_features:
		features['V_%s' % word] = (word in document_words)
	return features

In [65]:
# combine positive and negative tweets
pos_tweets = [(document, 'positive') for document in positive_word_doc]
neg_tweets = [(document, 'negative') for document in negative_word_doc]
all_tweets = pos_tweets + neg_tweets

In [66]:
#randomly shuffling the tweets
random.shuffle(all_tweets)

all_tweets[:10]

[('broken luggage mild case food poisoning ua lhr-iah. email this?',
  'negative'),
 ('guys please give update? sitting tarmac flight yet website still says time?',
  'negative'),
 ('flying las vegas chicago flight late flight announcement made', 'negative'),
 ('view downtown los angeles, hollywood sign, beyond rain mountains!',
  'positive'),
 ('really ashamed. entire business class cabin empty almost first class...',
  'negative'),
 ('- thank you!', 'positive'),
 ('charged one way baggage.find others leaving pittsburgh equipment charged hung me!!',
  'negative'),
 ('wife &amp; infant daughter aa abi-dfw. en route delayed. y’all hold aa rdu connect?',
  'negative'),
 ('yet staff philadelphia failed send luggage home me. #alwayshappensthere #angrytraveler',
  'negative'),
 ('prepared weather. want find take train phl catch connection.', 'negative')]

In [117]:
#BOW

In [118]:
# get all words from all tweets
all_words_list = [word for (document, sentiment) in all_tweets for word in document.split()]
all_words_list[:10]

['broken',
 'luggage',
 'mild',
 'case',
 'food',
 'poisoning',
 'ua',
 'lhr-iah.',
 'email',
 'this?']

In [119]:
all_words = nltk.FreqDist(all_words_list)

In [120]:
# get the 2500 most frequently appearing keywords in the corpus
word_items = all_words.most_common(2500)
word_features = [word for (word,count) in word_items]


In [71]:
# remove punctuations from word_features
pattern = re.compile(r'\W')
word_features = [re.sub(pattern, '', word) for word in word_features]


# remove '' from word_features
word_features = [word for word in word_features if word != '']
# remove single letter words from word_features
word_features = [word for word in word_features if len(word) > 1]

print(word_features[:50])


['flight', 'get', 'cancelled', 'customer', 'service', 'thanks', 'im', 'amp', 'thank', 'hold', 'still', 'us', 'hours', 'time', 'cant', 'plane', 'one', 'would', 'call', 'delayed', 'late', 'help', 'got', 'hour', 'gate', 'need', 'flights', 'back', 'like', 'flightled', 'please', 'bag', 'waiting', 'flight', 'trying', 'phone', 'never', 'ive', 'fly', 'guys', 'even', 'wait', 'going', 'airline', 'last', 'great', 'make', 'told', 'another', 'people']
2431


In [72]:
print(len(word_features))

2431


In [75]:
# extract features from all tweets
bow_featureset = [(document_features(d, word_features), c) for (d,c) in all_tweets]
X_bow = [d for (d,c) in all_tweets]
Y_bow = [c for (d,c) in all_tweets]


In [76]:
cv = CountVectorizer(binary=True, max_features=2000)

model = Pipeline([('cv', cv), ('nb', MultinomialNB())])

In [77]:
BOW_vector = cv.fit(X_bow)
BOW_vector.get_feature_names()[:100]

['aa',
 'able',
 'about',
 'absolute',
 'absolutely',
 'absurd',
 'abt',
 'accept',
 'acceptable',
 'accepted',
 'accepting',
 'access',
 'accommodate',
 'accommodating',
 'accommodations',
 'according',
 'account',
 'accountability',
 'acct',
 'accurate',
 'across',
 'act',
 'actions',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'additional',
 'address',
 'addressed',
 'admirals',
 'advance',
 'advantage',
 'advise',
 'advisory',
 'afford',
 'after',
 'afternoon',
 'again',
 'age',
 'agent',
 'agents',
 'ago',
 'ah',
 'ahead',
 'ahold',
 'air',
 'aircraft',
 'airline',
 'airlines',
 'airplane',
 'airport',
 'airports',
 'airways',
 'albany',
 'alert',
 'all',
 'alliance',
 'allow',
 'allowed',
 'allowing',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'alternate',
 'although',
 'always',
 'am',
 'amazing',
 'america',
 'american',
 'americanairlines',
 'amount',
 'amp',
 'and',
 'angry',
 'announce',
 'announced',
 'announcement',
 'announcements',
 'annoyed',
 'annoy

In [82]:
from sklearn.model_selection import cross_validate
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(model, X_bow, Y_bow, return_estimator=True,
                        scoring = scoring)

In [85]:
print("Accuracy : ")
print(scores['test_accuracy'])
print("Precision : ")
print(scores['test_precision_macro'])
print("Recall : ")
print(scores['test_recall_macro'])
print("F1 : ")
print(scores['test_f1_macro'])

Accuracy : 
[0.90428757 0.90294627 0.90511265 0.90901213 0.91507799]
Precision : 
[0.85359122 0.85706053 0.85912356 0.86668804 0.87375925]
Recall : 
[0.85192267 0.837476   0.8435596  0.84784234 0.86185689]
F1 : 
[0.85275276 0.84670181 0.85098628 0.8567573  0.8676102 ]


In [86]:
model.fit(X_bow, Y_bow)

Pipeline(steps=[('cv', CountVectorizer(binary=True, max_features=2000)),
                ('nb', MultinomialNB())])

In [87]:
model.predict(['that was a great flight'])

array(['positive'], dtype='<U8')

In [88]:
# SubjectivityList 

In [89]:
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}

    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    for word in document_words:
        if word in SL.keys():
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)  
        else:
            features['negativecount'] = 0
            features['positivecount'] = 0    
    return features

In [90]:
# this function returns a dictionary where you can look up words and get back 
#     the four items of subjectivity information described above
def readSubjectivity(path):
    flexicon = open(path, 'r')
    # initialize an empty dictionary
    sldict = { }
    for line in flexicon:
        fields = line.split()   # default is to split on whitespace
        # split each field on the '=' and keep the second part as the value
        strength = fields[0].split("=")[1]
        word = fields[2].split("=")[1]
        posTag = fields[3].split("=")[1]
        stemmed = fields[4].split("=")[1]
        polarity = fields[5].split("=")[1]
        if (stemmed == 'y'):
            isStemmed = True
        else:
            isStemmed = False
        # put a dictionary entry with the word as the keyword
        #     and a list of the other values
        sldict[word] = [strength, posTag, isStemmed, polarity]
    return sldict

In [91]:
# create a path to where the subjectivity file resides on your disk
SLpath = "subjclueslen1-HLTEMNLP05.tff"

# copy the readSubjectivity function into your session and cal the fn
SL = readSubjectivity(SLpath)
SL

{'abandoned': ['weaksubj', 'adj', False, 'negative'],
 'abandonment': ['weaksubj', 'noun', False, 'negative'],
 'abandon': ['weaksubj', 'verb', True, 'negative'],
 'abase': ['strongsubj', 'verb', True, 'negative'],
 'abasement': ['strongsubj', 'anypos', True, 'negative'],
 'abash': ['strongsubj', 'verb', True, 'negative'],
 'abate': ['weaksubj', 'verb', True, 'negative'],
 'abdicate': ['weaksubj', 'verb', True, 'negative'],
 'aberration': ['strongsubj', 'noun', False, 'negative'],
 'abhor': ['strongsubj', 'verb', True, 'negative'],
 'abhorred': ['strongsubj', 'adj', False, 'negative'],
 'abhorrence': ['strongsubj', 'noun', False, 'negative'],
 'abhorrent': ['strongsubj', 'adj', False, 'negative'],
 'abhorrently': ['strongsubj', 'anypos', False, 'negative'],
 'abhors': ['strongsubj', 'noun', False, 'negative'],
 'abidance': ['strongsubj', 'noun', False, 'positive'],
 'abide': ['strongsubj', 'anypos', True, 'positive'],
 'abject': ['strongsubj', 'adj', False, 'negative'],
 'abjectly': ['

In [92]:
all_tweets[0][0]

'broken luggage mild case food poisoning ua lhr-iah. email this?'

In [96]:
SL_featuresets = [(SL_features(d.split(" "), cv.get_feature_names(), SL), c) for (d, c) in all_tweets]
X_SL = [[v['negativecount'], v['positivecount']] for (v,k) in SL_featuresets]
Y_SL = [k for (v,k) in SL_featuresets]
X_SL[:7]


[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

In [95]:
Y_SL[:7]

['negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [97]:
model = MultinomialNB()
scores = cross_validate(model, X_SL, Y_SL, return_estimator=True,
                        scoring = scoring)
print("Accuracy : ")
print(scores['test_accuracy'])
print("Precision : ")
print(scores['test_precision_macro'])
print("Recall : ")
print(scores['test_recall_macro'])
print("F1 : ")
print(scores['test_f1_macro'])


Accuracy : 
[0.7955825  0.79462738 0.79332756 0.79462738 0.79506066]
Precision : 
[0.65663052 0.64180551 0.62708028 0.64399066 0.65026478]
Recall : 
[0.51596736 0.5199171  0.51752613 0.51541958 0.51647666]
F1 : 
[0.48158564 0.49148345 0.48741558 0.48113381 0.48310784]


In [98]:
train_set, test_set = SL_featuresets[:int(len(SL_featuresets)*0.8):], SL_featuresets[:int(len(SL_featuresets)*0.8)]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7962521663778163

In [99]:
classifier.show_most_informative_features(30)

Most Informative Features
           positivecount = 6              positi : negati =      6.3 : 1.0
           positivecount = 5              positi : negati =      5.5 : 1.0
           positivecount = 7              positi : negati =      3.9 : 1.0
           positivecount = 8              positi : negati =      3.9 : 1.0
           negativecount = 4              negati : positi =      3.8 : 1.0
           negativecount = 3              negati : positi =      3.5 : 1.0
           positivecount = 4              positi : negati =      2.9 : 1.0
           positivecount = 2              positi : negati =      2.3 : 1.0
           negativecount = 2              negati : positi =      2.0 : 1.0
           positivecount = 1              negati : positi =      1.9 : 1.0
           positivecount = 3              positi : negati =      1.9 : 1.0
           negativecount = 1              negati : positi =      1.8 : 1.0
           positivecount = 0              negati : positi =      1.1 : 1.0

In [100]:
# Negation Word feature

In [101]:
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = False
        features['V_NOT{}'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['V_NOT{}'.format(document[i])] = (document[i] in word_features)
        else:
            features['V_{}'.format(word)] = (word in word_features)
    return features

In [103]:
# this list of negation words 
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']
NOT_featuresets = [(NOT_features(d.split(" "), cv.get_feature_names(), negationwords), c) for (d, c) in all_tweets]


train_set, test_set = NOT_featuresets[int(len(NOT_featuresets)*0.8):], NOT_featuresets[:int(len(NOT_featuresets)*0.8)]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.8648180242634316

In [104]:
classifier.show_most_informative_features(30)

Most Informative Features
                 V_thank = True           positi : negati =     25.3 : 1.0
                  V_you! = False          positi : negati =     22.9 : 1.0
               V_awesome = True           positi : negati =     22.0 : 1.0
               V_forward = True           positi : negati =     22.0 : 1.0
               V_amazing = True           positi : negati =     19.4 : 1.0
                 V_kudos = True           positi : negati =     16.8 : 1.0
                    V_:) = False          positi : negati =     14.5 : 1.0
                 V_loved = True           positi : negati =     14.2 : 1.0
               V_thanks! = False          positi : negati =     13.8 : 1.0
                  V_love = True           positi : negati =     12.3 : 1.0
                   V_win = True           positi : negati =     11.7 : 1.0
              V_service! = False          positi : negati =     11.7 : 1.0
               V_delayed = True           negati : positi =     11.1 : 1.0

In [106]:
X_Not = np.array([np.array(list(v.values())) for (v,k) in NOT_featuresets])
Y_Not = [k for (v,k) in NOT_featuresets]
print(X_Not.shape)
X_Not[:10]

(11541,)


<ipython-input-106-9934b6f9424e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_Not = np.array([np.array(list(v.values())) for (v,k) in NOT_featuresets])


array([array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([ True, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False]),
       array([False, False, False, ..., False, False, False])],
      dtype=object)

In [107]:
num_folds = 3
subset_size = int(len(NOT_featuresets)/num_folds)
avg_f1 = 0
for i in range(num_folds):
    testing_this_round = NOT_featuresets[i*subset_size:(i+1)*subset_size]
    training_this_round = NOT_featuresets[:i*subset_size] + NOT_featuresets[(i+1)*subset_size:]
    classifier = nltk.NaiveBayesClassifier.train(training_this_round)
    y_pred = classifier.classify_many([v for (v,k) in testing_this_round])
    y_actual = [k for (v,k) in testing_this_round]
    print("Round ", i+1, " accuracy: ", nltk.classify.accuracy(classifier, testing_this_round))
    cm = nltk.ConfusionMatrix(y_pred, y_actual)
    # compute precision and recall from the confusion matrix
    labels = ['negative', 'positive']
    truePos = 0
    falsePos = 0
    falseNeg = 0

    for i in labels:
        for j in labels:
            if i==j:
                truePos += cm[i,j]
            else:
                falseNeg += cm[i,j]
                falsePos += cm[j,i]
    print("Precision: ", truePos/(truePos+falsePos))
    print("Recall: ", truePos/(truePos+falseNeg))
    print("F1: ", 2*truePos/(2*truePos+falsePos+falseNeg))
    print("\n")
    avg_f1 += 2*truePos/(2*truePos+falsePos+falseNeg)
    

Round  1  accuracy:  0.8884845334026514
Precision:  0.8884845334026514
Recall:  0.8884845334026514
F1:  0.8884845334026514


Round  2  accuracy:  0.8908240187158825
Precision:  0.8908240187158825
Recall:  0.8908240187158825
F1:  0.8908240187158825


Round  3  accuracy:  0.8996620743436444
Precision:  0.8996620743436444
Recall:  0.8996620743436444
F1:  0.8996620743436444




In [108]:
print("Average F1: ", avg_f1/num_folds)

Average F1:  0.8929902088207261


In [114]:
df = pd.read_csv('mycovid.csv')
df.head()

,organizations,uuid,thread,author,url,ord_in_thread,title,locations,entities,highlightText,language,persons,text,external_links,published,crawled,highlightTitle
0,[],0ad32fb9226b172b960cba62027cd8be7d9dfa22,"{'social': {'gplus': {'shares': 0}, 'pinterest...",Udayavani,https://m.dailyhunt.in/news/india/english/uday...,0,"Karnataka: Helplines, isolation wards set up f...",[],"{'persons': [], 'locations': [{'name': 'karnat...",NaN,english,[],Bengaluru: Isolation wards in hospitals across...,[],2020-01-31T11:46:00.000+02:00,2020-01-31T17:18:04.007+02:00,NaN
1,[],8c3c1e13471a5e6ef7e115588ab27f308285cf2e,"{'social': {'gplus': {'shares': 0}, 'pinterest...",NaN,http://cnnphilippines.com/videos/2020/1/28/Hea...,0,Health dept. monitoring 24 people for possible...,[],"{'persons': [], 'locations': [{'name': 'hubei'...",NaN,english,[],The government making sure that the new corona...,[],2020-01-28T02:00:00.000+02:00,2020-01-31T06:38:31.000+02:00,NaN
2,[],6333601dda63a855f38435ff80ccb89b8ed8c76a,"{'social': {'gplus': {'shares': 0}, 'pinterest...",jmccorm,https://news.ycombinator.com/item?id=22194798#...,88,NaN,[],"{'persons': [], 'locations': [], 'organization...",NaN,english,[],Apart from more people falling sick (as bad as...,[],2020-01-31T06:38:00.000+02:00,2020-01-31T06:39:09.006+02:00,NaN
3,[],4d639ca00ddb638ae57da43712aa50e0e56df2be,"{'social': {'gplus': {'shares': 0}, 'pinterest...",rttnews.com,https://www.rttnews.com/3064615/asian-markets-...,0,Asian Markets Mostly Higher,[],"{'persons': [{'name': 'santos', 'sentiment': '...",NaN,english,[],Asian stock markets are mostly higher on Frida...,[],2020-01-31T05:53:00.000+02:00,2020-01-31T20:00:27.007+02:00,NaN
4,[],624facc68c564b015fcab9a88ed7aaf8ea1e50da,"{'social': {'gplus': {'shares': 0}, 'pinterest...",Joe Easton,http://www.bnnbloomberg.ca/tesla-soars-as-bear...,0,Tesla soars as bearish analysts left with litt...,[],"{'persons': [{'name': 'tesla', 'sentiment': 'n...",NaN,english,[],"Cash flow was also “very strong,” at more than...",[],2020-01-30T02:00:00.000+02:00,2020-01-31T02:55:14.004+02:00,NaN


In [116]:
model.fit(x_bow,y_bow)

NameError: name 'x_bow' is not defined

In [115]:
df['Sentiment'] = df['text'].apply(lambda x: model.predict([x])[0])

NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
df.head()

In [ ]:
df['Sentiment_Not'] = df['text'].apply(lambda x: classifier.classify(NOT_features(x.split(" "), cv.get_feature_names(), negationwords)))

In [ ]:
# Print complete sentences of positive tweets
for index, row in df_pos.iterrows():
    if index>100:
        break
    print(row['text'])

In [ ]:
for index, row in df_neg.iterrows():
    if index>100:
        break
    print(row['text'])

In [ ]:
# Top 50 adjectives in positive tweets
pos_doc = [d.split(" ") for d in df_pos['text']]
adj_words = []
for doc in pos_doc:
    for word in doc:
        pos = wn.synsets(word, pos='a')
        if len(pos) > 0:
            adj_words.append(word)

pos_adjectives = pd.Series(adj_words).value_counts()[:50]

In [ ]:

print(', '.join(["'{}'".format(word) for word in pos_adjectives.index]))

In [ ]:
# Top 50 adjectives in negative tweets
neg_doc = [d.split(" ") for d in df_neg['text']]
adj_words = []
for doc in neg_doc:
    for word in doc:
        pos = wn.synsets(word, pos='a')
        if len(pos) > 0:
            adj_words.append(word)

neg_adjectives = pd.Series(adj_words).value_counts()[:50]



In [ ]:
print(', '.join(["'{}'".format(word) for word in neg_adjectives.index]))